In [1]:
#Import dependencies
import pandas as pd
import gmaps
import requests
from config import g_key

# Configure gmaps to use your Google API key.

gmaps.configure(api_key=g_key)

In [2]:
city_data_df=pd.read_csv("Challenge/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches,Snow inches
0,0,Gombong,ID,2020-02-28 04:14:32,-7.61,109.51,91.85,59,54,1.66,Clouds,0.0,0.0
1,1,Ushuaia,AR,2020-02-28 04:14:32,-54.80,-68.30,46.40,81,40,36.91,Clouds,0.0,0.0
2,2,Bethel,US,2020-02-28 04:14:33,41.37,-73.41,33.80,42,20,17.22,Clouds,0.0,0.0
3,3,Salinopolis,BR,2020-02-28 04:14:33,-0.61,-47.36,80.73,76,93,14.81,Clouds,0.0,0.0
4,4,Klaksvik,FO,2020-02-28 04:14:33,62.23,-6.59,33.80,64,100,9.17,Clouds,0.0,0.0


In [3]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

# Ask the customer the following:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_true = input("Do you want it to be raining? (yes/no) ")
snow_true = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 0
What is the maximum temperature you would like for your trip? 10
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [4]:
if rain_true == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Current Description"] == "Rain")]
elif snow_true == "yes": 
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Current Description"] == "Snow")]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches,Snow inches
8,8,Nome,US,2020-02-28 04:14:35,64.50,-165.41,1.40,77,1,0.98,Clear,0.0,0.00
33,33,Khatanga,RU,2020-02-28 04:14:45,71.97,102.50,0.10,82,71,8.43,Clouds,0.0,0.00
35,35,Umea,SE,2020-02-28 04:14:46,63.83,20.26,8.60,84,46,4.70,Clouds,0.0,0.00
45,45,Kauhajoki,FI,2020-02-28 04:14:49,62.43,22.18,8.01,84,0,3.69,Clear,0.0,0.00
61,61,Baruun-Urt,MN,2020-02-28 04:14:56,46.68,113.28,7.21,80,0,7.74,Clear,0.0,0.00
69,69,Pevek,RU,2020-02-28 04:10:55,69.70,170.31,8.65,88,100,8.10,Snow,0.0,0.38
77,77,Lac Du Bonnet,CA,2020-02-28 04:12:34,50.25,-96.06,8.01,86,100,0.83,Clouds,0.0,0.00
86,86,Yugorsk,RU,2020-02-28 04:15:05,61.31,63.33,9.25,91,0,4.50,Clear,0.0,0.00
95,95,Batagay-Alyta,RU,2020-02-28 04:15:09,67.80,130.41,5.92,66,0,4.85,Clear,0.0,0.00
119,119,Turukhansk,RU,2020-02-28 04:15:18,65.82,87.98,8.28,78,0,13.33,Clear,0.0,0.00


In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates and add Hotel Name
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#Searching for a hotel
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][1]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No Hotel around found"
        pass
    
output_data_file = "Challenge/WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file, index_label="City_ID")

hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Nome,US,1.40,Clear,64.50,-165.41,Red Moose
33,Khatanga,RU,0.10,Clouds,71.97,102.50,"""Khatanga"""
35,Umea,SE,8.60,Clouds,63.83,20.26,Scandic Plaza Umeå
45,Kauhajoki,FI,8.01,Clear,62.43,22.18,Kauhajoen Krouvi
61,Baruun-Urt,MN,7.21,Clear,46.68,113.28,tansag hotel
69,Pevek,RU,8.65,Snow,69.70,170.31,Munitsipal'naya Gostinitsa
77,Lac Du Bonnet,CA,8.01,Clouds,50.25,-96.06,The Lakeview Inn
86,Yugorsk,RU,9.25,Clear,61.31,63.33,Guest House Sibirskaya
95,Batagay-Alyta,RU,5.92,Clear,67.80,130.41,No Hotel around found
119,Turukhansk,RU,8.28,Clear,65.82,87.98,No Hotel around found


In [6]:
#Template for the pop-up marker
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""


hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Calling the figure
fig

Figure(layout=FigureLayout(height='420px'))